In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
def direction(val):
    val1 = Conversion().convert_int256_bytes_to_int(val, signed=False) 
    val2 = Conversion().convert_int256_bytes_to_int(val, signed=True)
    return val1 != val2

In [3]:
chain = Net.POLYGON
platform = Platform.UNIV3
contract = JSONContract.UniswapV3Factory

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [4]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
pool_created_filt = Filter.create_filter(address=None, event_types=[Pair.events.PoolCreated])  # Listen events from any smart contract

In [5]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 25000

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=pool_created_filt)

In [8]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    event_signature, token0, token1, fee = evt["topics"]
    args = Conversion().decode_data(evt["data"])

    token0_address = Conversion().convert_uint256_string_to_address(token0)
    token1_address = Conversion().convert_uint256_string_to_address(token1)
    token0 = Token().fetch_erc20_details(web3, token0_address, raise_on_error=False)
    token1 = Token().fetch_erc20_details(web3, token1_address, raise_on_error=False)
    
    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = token0_address
    event['details']['token1'] = token1_address
    event['details']['token0_symbol'] = token0.symbol
    event['details']['token1_symbol'] = token1.symbol  
    event['details']['fee'] = Conversion().convert_uint256_string_to_int(fee) 
 
    events[k] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Swap at block:60,333,849 tx:0x02f6438a03e818bc241b9fd2e21efd41f4bdc9c8cf5234d23a53141555a6368d
Swap at block:60,334,850 tx:0xa1e7e340c0743f10bdab9bea52ac77e99050aef2a40b0257b15c26e2205db277
Swap at block:60,335,561 tx:0x6c6337ef770f76dd758400f69ef5ac7b8fe8e3f8f16663db09076f8e35820090
Swap at block:60,337,086 tx:0x364ca7938e14363e1ff327da60699ce37a086223e1260e0826ef20043950a1e1
Swap at block:60,337,693 tx:0xda43d766c32f352e4e1037e88b6b8bb6bd47286db4c2f9eee21cacf5a70b4b33
Swap at block:60,338,209 tx:0x9d263ff533413b77fd7a22141f4d96375a231a0da350d5af71291f26eaba0fcf
Swap at block:60,339,494 tx:0x0501e7424db570f6af472f922134e73d65eaffbe6a73ae7797c0d8f514b05206
Swap at block:60,342,207 tx:0x1e6328cf821f55a26fa817bcfadc9e362cdc7484086c3f75c0b645235330ec3d
Swap at block:60,345,067 tx:0x2d4c6de6c59a0f7031d57dfd117b8dc9fc9e44be9cda8152d535444657e93bf9
Swap at block:60,346,600 tx:0xddba33b8ff69c9d47bd06c18660c5372f42b7cb0d231fd966debb80d90d7c590
Swap at block:60,352,943 tx:0xf60e4783bc9d78f626a9

In [9]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x02f6438a03e818bc241b9fd2e21efd41f4bdc9c8cf52...,60333849,1723107989,{'web3_type': <class 'web3._utils.datatypes.Po...
1,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0xa1e7e340c0743f10bdab9bea52ac77e99050aef2a40b...,60334850,1723110125,{'web3_type': <class 'web3._utils.datatypes.Po...
2,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x6c6337ef770f76dd758400f69ef5ac7b8fe8e3f8f166...,60335561,1723111635,{'web3_type': <class 'web3._utils.datatypes.Po...
3,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x364ca7938e14363e1ff327da60699ce37a086223e126...,60337086,1723114875,{'web3_type': <class 'web3._utils.datatypes.Po...
4,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0xda43d766c32f352e4e1037e88b6b8bb6bd47286db4c2...,60337693,1723116165,{'web3_type': <class 'web3._utils.datatypes.Po...
5,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x9d263ff533413b77fd7a22141f4d96375a231a0da350...,60338209,1723117265,{'web3_type': <class 'web3._utils.datatypes.Po...
6,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x0501e7424db570f6af472f922134e73d65eaffbe6a73...,60339494,1723119995,{'web3_type': <class 'web3._utils.datatypes.Po...
7,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x1e6328cf821f55a26fa817bcfadc9e362cdc7484086c...,60342207,1723125759,{'web3_type': <class 'web3._utils.datatypes.Po...
8,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x2d4c6de6c59a0f7031d57dfd117b8dc9fc9e44be9cda...,60345067,1723131850,{'web3_type': <class 'web3._utils.datatypes.Po...
9,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0xddba33b8ff69c9d47bd06c18660c5372f42b7cb0d231...,60346600,1723135138,{'web3_type': <class 'web3._utils.datatypes.Po...


In [10]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv3factory',
  'type': 'poolcreated',
  'platform': 'uniswap_v3',
  'address': '0x1f98431c8ad98523631ae4a59f267346ea31f984',
  'tx_hash': '0x02f6438a03e818bc241b9fd2e21efd41f4bdc9c8cf5234d23a53141555a6368d',
  'blk_num': 60333849,
  'timestamp': 1723107989,
  'details': {'web3_type': web3._utils.datatypes.PoolCreated,
   'token0': '0x1F45a952985e952bE333Bcb908fcC06300FD5EEe',
   'token1': '0xc2132D05D31c914a87C6611C10748AEb04B58e8F',
   'token0_symbol': 'GMD',
   'token1_symbol': 'USDT',
   'fee': 100}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv3factory',
  'type': 'poolcreated',
  'platform': 'uniswap_v3',
  'address': '0x1f98431c8ad98523631ae4a59f267346ea31f984',
  'tx_hash': '0xa1e7e340c0743f10bdab9bea52ac77e99050aef2a40b0257b15c26e2205db277',
  'blk_num': 60334850,
  'timestamp': 1723110125,
  'details': {'web3_type': web3._utils.datatypes.PoolCreated,
   'token0': '0x2D841F3E0FBBaB0466A5bd655A33Ce15336b7B3C',
   'token1': '0x432